In [1]:
!pip -q install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.0 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.

In [2]:
TRAIN_PATH = "/kaggle/input/dataset-with-vnese-output/train_df_with_VNESE_output.csv"
VAL_PATH = "/kaggle/input/dataset-with-vnese-output/val_df_with_VNESE_output.csv"
TEST_PATH = "/kaggle/input/dataset-with-vnese-output/test_df_with_VNESE_output.csv"

In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": TRAIN_PATH, "val": VAL_PATH, "test": TEST_PATH})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_comment', 'aspect', 'sentiments', 'VNESE_text_output'],
        num_rows: 2961
    })
    val: Dataset({
        features: ['clean_comment', 'aspect', 'sentiments', 'VNESE_text_output'],
        num_rows: 1290
    })
    test: Dataset({
        features: ['clean_comment', 'aspect', 'sentiments', 'VNESE_text_output'],
        num_rows: 500
    })
})

Thêm phần instruction cho các câu review


In [5]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("VietAI/vit5-base")

# Tokenize the inputs and targets
def preprocess(example):
    model_input = tokenizer(
        example['clean_comment'], 
        padding="max_length", 
        truncation=True, 
        max_length=256
    )
    labels = tokenizer(
        text_target=example['VNESE_text_output'], 
        padding="max_length", 
        truncation=True, 
        max_length=128
    )
    model_input['labels'] = labels['input_ids']
    return model_input

## phan nay co them instruction nhung ket qua kem hon
# def preprocess(batch):
#     input_texts = [
#         f"Hãy rút trích các cặp thể loại và trạng thái trong bình luận sau: {c}"
#         for c in batch["clean_comment"]
#     ]
#     labels_texts = batch["VNESE_text_output"]

#     model_inputs = tokenizer(
#         input_texts,
#         padding="max_length",
#         truncation=True,
#         max_length=256,
#     )
#     labels = tokenizer(
#         labels_texts,
#         padding="max_length",
#         truncation=True,
#         max_length=128,
#     )

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

tokenized_dataset = dataset.map(
    preprocess, 
    batched=True,
)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/2961 [00:00<?, ? examples/s]

Map:   0%|          | 0/1290 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [6]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("VietAI/vit5-base")

2025-05-23 07:06:08.255245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747983968.446768      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747983968.500873      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [7]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# training_args = TrainingArguments(
#     output_dir="./t5-finetuned",
#     run_name="t5-finetune-v1",
#     eval_strategy="steps",
#     eval_steps=500,
#     save_steps=1000,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     greater_is_better=False,
#     report_to="none",  
# )

training_args = TrainingArguments(
    output_dir="./t5-finetuned",
    run_name="t5-finetune-v1",
    eval_strategy="steps",
    eval_steps=500,  # more frequent evaluation
    save_steps=500,
    per_device_train_batch_size=8,  # larger batch size
    per_device_eval_batch_size=8,
    lr_scheduler_type="cosine",
    num_train_epochs=15,            # more epochs
    learning_rate=2e-5,             # lower learning rate
    fp16=True,                      # mixed precision
    weight_decay=0.01,              # regularization
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
)


from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,0.620600,0.040023
1000,0.043000,0.035598
1500,0.031800,0.036571
2000,0.022000,0.039465
2500,0.017000,0.042989


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2500, training_loss=0.14688394584655762, metrics={'train_runtime': 1282.1607, 'train_samples_per_second': 34.641, 'train_steps_per_second': 4.34, 'total_flos': 6076790789898240.0, 'train_loss': 0.14688394584655762, 'epoch': 6.738544474393531})

In [8]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

comment = """
Nhà hàng này món ăn cực kỳ ngon và lạ nữa. Không gian đẹp, nhẹ nhàng và tinh tế. Phục vụ cũng rất ok, giá thì hơi cao nhưng so với mặt bằng Q. 1 thì giá như thế là phù hợp rồi.  chả cá chiên thuyền buồm 199k.  Xôi đùi gà quay 139k.  Black Currant 50k.  Coronarita 199k.  Xôi ghẹ nè...
"""

input_text = f"Hãy rút trích các cặp thể loại và trạng thái trong bình luận sau: {comment}"
input_ids = tokenizer.encode(input_text,return_tensors="pt").to(device)

output = model.generate(input_ids, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

chất lượng đồ ăn tốt và không gian tốt và phục vụ tốt và giá tiền món ăn tạm


# Predict on test dataset


In [9]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(tokenized_dataset['test'], batch_size=8)


predictions = []

for batch in test_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    
    # with torch.no_grad():
    #     generated_ids = model.generate(
    #         input_ids=input_ids,
    #         attention_mask=attention_mask,
    #         max_length=128,        # same as your target length
    #         num_beams=4,           # optional: beam search for better results
    #         early_stopping=True
    #     )

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=6,
            top_k=50,
            temperature=0.7,
            early_stopping=True
        )
    
    preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    predictions.extend(preds)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [10]:
# Convert output VNESE back to format that can be evaluated
def convert_output_to_list(output_text, aspect_category_list):
    # Create a dictionary to map sentiment words to numbers
    sentiment_map = {
        'tốt': 1,
        'tệ': 2,
        'tạm': 3
    }
    
    # Initialize the result list with 0 (not mentioned)
    result = [0] * len(aspect_category_list)
    
    # Split the output by ' và ' to get each aspect-sentiment pair
    pairs = output_text.split(' và ')
    
    for pair in pairs:
        # Find the matching sentiment
        for sentiment_word, sentiment_value in sentiment_map.items():
            if sentiment_word in pair:
                aspect_name = pair.replace(' ' + sentiment_word, '').strip()
                if aspect_name in aspect_category_list:
                    index = aspect_category_list.index(aspect_name)
                    result[index] = sentiment_value
                break  # No need to check other sentiments for this pair
    
    return result

aspect_category_list = [
    'không gian', 'giá tiền đồ uống', 'chất lượng đồ uống', 'lựa chọn đồ uống',
    'giá tiền món ăn', 'chất lượng đồ ăn', 'lựa chọn đồ ăn', 'địa chỉ', 'nhà hàng nói chung', 
    'vấn đề khác', 'giá cả nhà hàng', 'phục vụ'
]

output_text = "chất lượng đồ ăn tốt và lựa chọn đồ ăn tốt và không gian tốt và giá tiền món ăn tạm"
result = convert_output_to_list(output_text, aspect_category_list)
print(result)


[1, 0, 0, 0, 3, 1, 1, 0, 0, 0, 0, 0]


In [11]:
import numpy as np

y_pred = []
for pred in predictions:
    y_pred.append(convert_output_to_list(pred,aspect_category_list))

# convert ground-truth y_test

y_test_raw_text = dataset['test']['VNESE_text_output']
y_test = []
for raw in y_test_raw_text:
    y_test.append(convert_output_to_list(raw,aspect_category_list))

y_pred = np.array(y_pred)
y_test = np.array(y_test)

# Evaluation Scikit-learn


In [12]:
# Evaluation
import sys
import pandas as pd
from tqdm import tqdm
from IPython.display import display_html
from sklearn.metrics import classification_report
import numpy as np

class PolarityMapping:
    INDEX_TO_POLARITY = { 0: None, 1: 'positive', 2: 'negative', 3: 'neutral' }
    # INDEX_TO_ONEHOT = { i: [1 if i == j else 0 for j in INDEX_TO_POLARITY] for i in INDEX_TO_POLARITY } 
    # POLARITY_TO_INDEX = { polarity: index for index, polarity in INDEX_TO_POLARITY.items() }
    INDEX_TO_ONEHOT = { 0: [1, 0, 0, 0], 1: [0, 1, 0, 0], 2: [0, 0, 1, 0], 3: [0, 0, 0, 1] }
    POLARITY_TO_INDEX = { None: 0, 'positive': 1, 'negative': 2, 'neutral': 3 }

sys.path.append('..')


class VLSP2018SklearnEvaluator:
    def __init__(self, y_test, y_pred, aspect_category_names): 
        aspect_cate_test, aspect_cate_pred = [], []
        aspect_cate_polar_test, aspect_cate_polar_pred = [], []

        for row_test, row_pred in zip(y_test, y_pred):
            for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
                aspect_cate_test.append(aspect_category_names[index] if col_test != 0 else 'None#None')
                aspect_cate_pred.append(aspect_category_names[index] if col_pred != 0 else 'None#None')
                aspect_cate_polar_test.append(aspect_category_names[index] + f',{PolarityMapping.INDEX_TO_POLARITY[col_test]}')
                aspect_cate_polar_pred.append(aspect_category_names[index] + f',{PolarityMapping.INDEX_TO_POLARITY[col_pred]}')

        self.aspect_cate_polar_report = classification_report(aspect_cate_polar_test, aspect_cate_polar_pred, output_dict=True, zero_division=1.0)
        self.aspect_cate_report = classification_report(aspect_cate_test, aspect_cate_pred, output_dict=True, zero_division=1.0)
        self.polarity_report = classification_report(y_test.flatten(), y_pred.flatten(), target_names=PolarityMapping.POLARITY_TO_INDEX, output_dict=True)
        self._merge_all_reports()
        self._build_macro_avg_df()
        
    
    def report(self, report_type='all'):
        if report_type.lower() == 'all': self._display_all_reports()
        elif report_type.lower() == 'aspect#category,polarity': return pd.DataFrame(self.aspect_cate_polar_report).T
        elif report_type.lower() == 'aspect#category': return pd.DataFrame(self.aspect_cate_report).T
        elif report_type.lower() == 'polarity': return pd.DataFrame(self.polarity_report).T
        elif report_type.lower() == 'macro_avg': return self.macro_avg_df()
        else: raise ValueError('report_type must be in ["all", "aspect#category,polarity", "aspect#category", "polarity", "macro_avg"]')
        
        
    def _merge_all_reports(self):
        self.merged_report = {}
        for key, metrics in self.aspect_cate_polar_report.items():
            # Check if key in the form of 'aspect#category,polarity' (Check if it's not 'accuracy' or 'macro avg' or 'weighted avg')
            if key in ['accuracy', 'macro avg', 'weighted avg']:
                self.merged_report[key] = {
                    'aspect#category': self.aspect_cate_report[key],
                    'aspect#category,polarity': metrics
                }
            else:
                aspect_cate, polarity = key.split(',')
                if aspect_cate not in self.merged_report:
                    self.merged_report[aspect_cate] = {'aspect#category': self.aspect_cate_report[aspect_cate]}
                self.merged_report[aspect_cate][polarity] = metrics
                
                
    def _build_macro_avg_df(self):
        self.macro_avg_df = pd.DataFrame([{
            'accuracy': f"{report['accuracy']:.3f}", 
            # **{metric: report['macro avg'][metric] for metric in report['macro avg'] if metric != 'accuracy'}
            'precision': f"{report['macro avg']['precision']:.3f}",
            'recall': f"{report['macro avg']['recall']:.3f}",
            'f1-score': f"{report['macro avg']['f1-score']:.3f}",
            'support': report['macro avg']['support']
        } for report in [self.aspect_cate_polar_report, self.aspect_cate_report, self.polarity_report]])
        self.macro_avg_df.index = ['Aspect#Category,Polarity', 'Aspect#Category', 'Polarity']
    
        
    def _display_all_reports(self):
        metric_names = list(self.merged_report.values())[0]['aspect#category']
        html_str = f"""
            <tr>
                <th style="font-weight: bold; text-align: center;" rowspan="2">ACSA Report (w/o "None" polarity)</th>
                <th style="font-weight: bold; text-align: center;" colspan="{len(metric_names)}">Aspect#Category</th>
                <th style="font-weight: bold; text-align: center;" colspan="{len(metric_names)}">Aspect#Category,Polarity</th>
            </tr>
            <tr>
                {''.join([f'<th>{metric_name}</th>' for metric_name in metric_names] * 2)}
            </tr>
        """

        for key, merged_metrics in tqdm(self.merged_report.items()):
            if key in ['accuracy', 'macro avg', 'weighted avg']: continue
            polarities = merged_metrics.keys() - {'aspect#category', 'None'}
            aspect_cate_html = ''.join(
                f'<td rowspan="{len(polarities)}">{value if name == "support" else f"{value:.3f}"}</td>'
                for name, value in self.merged_report[key]['aspect#category'].items()
            )
            for index, polarity in enumerate(polarities):
                aspect_cate_polar_html = ''.join(
                    f'<td>{value if name == "support" else f"{value:.3f}"}</td>'
                    for name, value in self.merged_report[key][polarity].items()
                )
                html_str += f"""
                    <tr>
                        <td>{key},{polarity}</td>
                        {aspect_cate_html if index == 0 else ''}
                        {aspect_cate_polar_html}
                    </tr>
                """

        display_html(f'''
            <div style="display: flex; align-items: flex-start; flex-wrap: nowrap">
                <table style="margin-right: 10px">{html_str}</table> 
                <div style="display: flex; align-items: center; flex-direction: column">
                    <b>Polarity Report</b><br>
                    {pd.DataFrame(self.polarity_report).T.to_html()}<br>
                    <b>Macro Avg Report</b><br>
                    {self.macro_avg_df.to_html()}
                </div>
            </div>
        ''', raw=True)      

In [13]:
ASPECT_CATEGORY_NAMES = ['AMBIENCE#GENERAL', 'DRINKS#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE&OPTIONS', 'FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE&OPTIONS', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL', 'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#PRICES', 'SERVICE#GENERAL']

In [14]:
sk_eval = VLSP2018SklearnEvaluator(y_test, y_pred, ASPECT_CATEGORY_NAMES)
# sk_eval.report(report_type='Aspect#Category,Polarity').round(3)
# sk_eval.report(report_type='Aspect#Category').round(3)
# sk_eval.report(report_type='Polarity').round(3)
# sk_eval.report(report_type='macro_avg').round(3)
sk_eval.report(report_type='all')

100%|██████████| 15/15 [00:00<00:00, 62601.55it/s]


ACSA Report (w/o "None" polarity) 
 Aspect#Category 
 Aspect#Category,Polarity 
 
 
 precision recall f1-score support precision recall f1-score support 
 
 
 
 AMBIENCE#GENERAL,negative 
 0.951 0.757 0.843 255 
 0.722 0.500 0.591 26 
 
 
 
 AMBIENCE#GENERAL,positive 
 
 0.845 0.812 0.828 181 
 
 
 
 AMBIENCE#GENERAL,neutral 
 
 0.727 0.167 0.271 48 
 
 
 
 DRINKS#PRICES,negative 
 0.862 0.329 0.476 76 
 0.000 0.000 0.000 3 
 
 
 
 DRINKS#PRICES,positive 
 
 0.538 0.250 0.341 28 
 
 
 
 DRINKS#PRICES,neutral 
 
 0.833 0.222 0.351 45 
 
 
 
 DRINKS#QUALITY,negative 
 0.912 0.437 0.590 71 
 0.333 0.167 0.222 6 
 
 
 
 DRINKS#QUALITY,positive 
 
 0.862 0.463 0.602 54 
 
 
 
 DRINKS#QUALITY,neutral 
 
 0.500 0.091 0.154 11 
 
 
 
 DRINKS#STYLE&OPTIONS,negative 
 0.574 0.587 0.581 46 
 0.200 1.000 0.333 1 
 
 
 
 DRINKS#STYLE&OPTIONS,positive 
 
 0.590 0.561 0.575 41 
 
 
 
 DRINKS#STYLE&OPTIONS,neutral 
 
 0.000 0.000 0.000 4 
 
 
 
 FOOD#PRICES,negative 
 0.812 0.937 0.870 331 
 0.560 0.500 0.528 28 
 
 
 
 FOOD#PRICES,positive 
 
 0.650 0.727 0.686 128 
 
 
 
 FOOD#PRICES,neutral 
 
 0.692 0.846 0.761 175 
 
 
 
 FOOD#QUALITY,negative 
 0.948 0.967 0.958 457 
 0.400 0.364 0.381 11 
 
 
 
 FOOD#QUALITY,positive 
 
 0.917 0.901 0.909 403 
 
 
 
 FOOD#QUALITY,neutral 
 
 0.350 0.488 0.408 43 
 
 
 
 FOOD#STYLE&OPTIONS,negative 
 0.912 0.903 0.908 403 
 0.714 0.312 0.435 16 
 
 
 
 FOOD#STYLE&OPTIONS,positive 
 
 0.805 0.829 0.817 334 
 
 
 
 FOOD#STYLE&OPTIONS,neutral 
 
 0.354 0.321 0.337 53 
 
 
 
 LOCATION#GENERAL,negative 
 0.871 0.676 0.761 179 
 0.533 0.500 0.516 16 
 
 
 
 LOCATION#GENERAL,positive 
 
 0.490 0.391 0.435 64 
 
 
 
 LOCATION#GENERAL,neutral 
 
 0.685 0.505 0.581 99 
 
 
 
 RESTAURANT#GENERAL,negative 
 0.638 0.610 0.624 223 
 0.667 0.154 0.250 13 
 
 
 
 RESTAURANT#GENERAL,positive 
 
 0.623 0.629 0.626 205 
 
 
 
 RESTAURANT#GENERAL,neutral 
 
 0.000 0.000 0.000 5 
 
 
 
 RESTAURANT#MISCELLANEOUS,negative 
 1.000 0.031 0.060 130 
 0.500 0.111 0.182 9 
 
 
 
 RESTAURANT#MISCELLANEOUS,positive 
 
 1.000 0.034 0.066 59 
 
 
 
 RESTAURANT#MISCELLANEOUS,neutral 
 
 1.000 0.000 0.000 62 
 
 
 
 RESTAURANT#PRICES,negative 
 0.600 0.082 0.145 73 
 1.000 0.000 0.000 5 
 
 
 
 RESTAURANT#PRICES,positive 
 
 0.500 0.114 0.185 44 
 
 
 
 RESTAURANT#PRICES,neutral 
 
 1.000 0.000 0.000 24 
 
 
 
 SERVICE#GENERAL,negative 
 0.863 0.754 0.805 175 
 0.640 0.640 0.640 25 
 
 
 
 SERVICE#GENERAL,positive 
 
 0.833 0.742 0.785 128 
 
 
 
 SERVICE#GENERAL,neutral 
 
 0.357 0.227 0.278 22 
 
 
 
 Polarity Report 
 
 
 
 
 precision 
 recall 
 f1-score 
 support 
 
 
 
 
 None 
 0.839837 
 0.919576 
 0.877899 
 3581.0000 
 
 
 positive 
 0.782523 
 0.713601 
 0.746474 
 1669.0000 
 
 
 negative 
 0.555556 
 0.408805 
 0.471014 
 159.0000 
 
 
 neutral 
 0.590909 
 0.439932 
 0.504365 
 591.0000 
 
 
 accuracy 
 0.801500 
 0.801500 
 0.801500 
 0.8015 
 
 
 macro avg 
 0.692206 
 0.620478 
 0.649938 
 6000.0000 
 
 
 weighted avg 
 0.791841 
 0.801500 
 0.793766 
 6000.0000 
 
 
 
 Macro Avg Report 
 
 
 
 
 accuracy 
 precision 
 recall 
 f1-score 
 support 
 
 
 
 
 Aspect#Category,Polarity 
 0.801 
 0.646 
 0.494 
 0.497 
 6000 
 
 
 Aspect#Category 
 0.847 
 0.829 
 0.615 
 0.654 
 6000 
 
 
 Polarity 
 0.801 
 0.692 
 0.620 
 0.650 
 6000

# Evaluation VLSP


In [15]:
def vlsp_save_as(save_path, raw_texts, encoded_review_labels, aspect_category_names):
        with open(save_path, 'w', encoding='utf-8') as file:
            for index, encoded_label in tqdm(enumerate(encoded_review_labels)):
                polarities = map(lambda x: PolarityMapping.INDEX_TO_POLARITY[x], encoded_label)
                acsa = ', '.join(
                    f'{{{aspect_category}, {polarity}}}' 
                    for aspect_category, polarity in zip(aspect_category_names, polarities) if polarity
                )
                file.write(f"#{index + 1}\n{raw_texts[index]}\n{acsa}\n\n")

In [16]:
MODEL_NAME = "restaurant-3"

In [17]:
test_reviews = []

TEST_PATH = "/kaggle/input/restaurant-test-text/3-VLSP2018-SA-Restaurant-test.txt"

with open(TEST_PATH, "r", encoding = 'utf-8') as file:
    lines = file.readlines()
    for i in range(1,len(lines),4):
        test_reviews.append(str(lines[i]).strip())

In [18]:
vlsp_save_as(f'/kaggle/working/{MODEL_NAME}.txt', test_reviews, y_pred, ASPECT_CATEGORY_NAMES)

500it [00:00, 93924.76it/s]


In [19]:
!java /kaggle/input/saevaluate/SAEvaluate.java /kaggle/input/restaurant-test-text/3-VLSP2018-SA-Restaurant-test.txt /kaggle/working/{MODEL_NAME}.txt

Row mismatch:﻿#1 <-> #1
Evaluation Result >> File:/kaggle/working/restaurant-3.txt<> [/kaggle/input/restaurant-test-text/3-VLSP2018-SA-Restaurant-test.txt]
                    Gold count	223	71	76	46	179	130	403	331	457	255	73	175
                  ANSWER count	213	34	29	47	139	4	399	382	466	203	10	153

        Correct ANSWER: aspect	135	30	24	26	121	4	364	310	442	193	6	132
             Precision: aspect	0.6338	0.8824	0.8276	0.5532	0.8705	1.0000	0.9123	0.8115	0.9485	0.9507	0.6000	0.8627
                Recall: aspect	0.6054	0.4225	0.3158	0.5652	0.6760	0.0308	0.9032	0.9366	0.9672	0.7569	0.0822	0.7543
              F1 score: aspect	0.6193	0.5714	0.4571	0.5591	0.7610	0.0597	0.9077	0.8696	0.9577	0.8428	0.1446	0.8049

  Over All ANSWER: aspect:----	0.8595	0.7387	0.7946

  Correct ANSWER: aspect,value	130	26	16	24	83	3	299	255	388	168	5	116
      Precision: aspect, value	0.6103	0.7647	0.5517	0.5106	0.5971	0.7500	0.7494	0.6675	0.8326	0.8276	0.5000	0.7582
         Recall: aspect, value	0.5830	

In [20]:
output = "/kaggle/working/savedmodel"
model.save_pretrained(output)
tokenizer.save_pretrained(output)

('/kaggle/working/savedmodel/tokenizer_config.json',
 '/kaggle/working/savedmodel/special_tokens_map.json',
 '/kaggle/working/savedmodel/spiece.model',
 '/kaggle/working/savedmodel/added_tokens.json')